<img src="https://drive.google.com/uc?id=181S0KcAdGeAajZ1apcbOvoh3wvYuKtzd">
<b>Autores</b>:

*   Aldo von Wangenheim (aldo.vw@ufsc.br)
*   Rodrigo de Paula e Silva Ribeiro (ribeiro.rodrigo@posgrad.ufsc.br)


CERTIFIQUE-SE DE TER INCIADO O AMBIENTE EM MODO <b>GPU</b> ANTES DE REALIZAR OS PROCEDIMENTOS.<br>
NA TROCA DO AMBIENTE ELE <b><u>PERDE</u></b> TODOS OS DADOS BAIXADOS.

AO RE-EXECUTAR OS PROCEDIMENTOS (ex. diminuir o batch size), <br>REINICIE O AMBIENTE PARA LIBERAR O CACHE DO PYTORCH (reiniciar não perde os dados da sessão).<br>(alt+m ou pelo menu "Ambiente de Execução")

EXECUTE UTILIZANDO RUN ALL ou CTRL+F9

<a href="https://colab.research.google.com/github/awangenh/vision/blob/master/jupyter/14.4.Detecção%20de%20Sinais%20de%20Trânsito%20-%20YOLOv10.ipynb"><img align="left"  src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>&nbsp; &nbsp;<a href=""><img align="left" src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/License-CC-BY-ND-4.0-orange.png" alt="Creative Commons 4.0 License" title="Creative Commons 4.0 License"></a>&nbsp; &nbsp; <a href=""><img align="left" src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/Jupyter-Notebook-v.1.0-blue.png" alt="Jupyter Version" title="Jupyter Version"></a>&nbsp; &nbsp;<a href=""><img align="left"  src="https://img.shields.io/badge/python-3.10-greeng" alt="Python Version" title="Python Version"></a>

In [ ]:
!nvidia-smi
!nvcc --version

# Definir a pasta base atual e Configurações GPU

In [ ]:
!nvcc --version

try:
    import google.colab
    _ON_COLAB = True
except:
    _ON_COLAB = False
    %reload_ext autoreload
    %autoreload 2
    %matplotlib inline
    from IPython import get_ipython
    
print('Running on Google Colab = ', _ON_COLAB)
import os
import torch

# Definir o caminho base
if _ON_COLAB:
    BASE_PATH = "/content"
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"
else:
    BASE_PATH = "/lapix" # trocar para o base path do seu computador se estiver rodando localmente, deixar /lapix se estiver rodando nos conteiners lapix
    VISIBLE_GPUS = [7] # Selecionar a GPU PARA RODAR! VER QUAL ESTA LIVRE

    if torch.cuda.device_count() != 8:
        print("GPU SETADA - PULANDO ETAPA")
    else:
        os.environ["CUDA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
        os.environ["NVIDIA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
        print("CUDA GPUS NUMBER: ", torch.cuda.device_count())
    
os.chdir(BASE_PATH) # garantir que está executando no caminho base definido

# O que é YOLO?
YOLO (“You Only Look Once”), é uma série de detectores de objetos de processamento rápido que vem sendo desenvolvida por diversos autores. Ao contrário dos detectores tradicionais que utilizam áreas de interesse para propor regiões (ROI) que podem conter um objeto na imagem (conhecidos como detectores de 2-estágios), a arquitetura YOLO divide a imagem em grids e vasculha a imagem como um todo (detector de um único estágio).

Neste notebook didático, é utilizado a versão 10 desta arquitetura.<br>
Algumas notas sobre a versão 10:<br>
* Em uma abordagem nova, foi removido a NMS e trocado por um head 1x1 para reduzir o peso computacional e melhorar a latência.
* Assim como a versão 9, a versão 10 é pré-treinada no [COCO Dataset](https://cocodataset.org/#home) e não no ImageNet.
* Diferente de outras versões anteriores (9, 7, 6, 4), a versão 10 é construida sobre o Yolo V8, no framework Ultralytics de modo que pode ser executada como módulo cli ou pela lib ultralytics, este notebook utiliza chamada de módulo cli.
* Foram realizadas diversas melhorias na arquitetura em relação a inferência, o mais notável é o consumo de VRAM, em comparação com Yolov9 e utilizando o mesmo modelo base, YoloV10 consome quase metade da VRAM, podendo rodar com BATCH maior que o YoloV9 nas mesmas configurações.

Diferentemente do Detectron2 (framework customizável), YOLO é uma arquitetura parametrizada e única, embora é possível alterar diversos parâmetros na rotina de treino oficial, muitas alterações ou valores muito diferentes podem descaracterizar a rede perdendo a sua capacidade de atingir resultados SOTA (state of the art).

Neste notebook apenas alguns parâmetros de performance são editáveis, para editar parâmetros mais específicos ou até mesmo implementar modificações, é recomendável ler e se familiarizar com a arquitetura.

Links:<br>
[Link Artigo](https://arxiv.org/abs/2405.14458) <br>
[Link git](https://github.com/THU-MIG/yolov10)


# Instalação do YoloV10 do repositório oficial e suas dependências.

In [ ]:
!git clone https://github.com/THU-MIG/yolov10.git
os.chdir(os.path.join(HOME_DIR, "yolov10"))
!pip install -r requirements.txt
!pip install -e .

# Montar o google drive
As 'runs' de treino serão salvas na pasta "aula_yolov10" no seu drive.

In [ ]:
from google.colab import drive
drive.mount(os.path.join(BASE_PATH, 'gdrive'), force_remount=False)

# Baixar o dataset de itens de trânsito já no formato YOLO
O dataset de itens de trânsito tem como objetivo didático demonstrar a possibilidade de identificar alguns itens comuns de trânsito.<br>
É um conjunto anotado no formato "bounding box" para detecção de objetos.<br>
O conjunto possui 4 classes distintas:
* trafficlight
* stop
* speedlimit
* crosswalk

Amostra de anotações:<br>
<img src="https://drive.google.com/uc?id=1AoC87UUaJ5v3W03mTE6gQocW6sYgCEuz">


In [ ]:
import os
os.chdir(BASE_PATH)
!gdown 1sIhCtoOMnpCXlByxDV95iRQDWjTPKr9v
!unzip -qq -u trafficsign.zip
!rm -rf trafficsign.zip


# Baixar o peso do modelo padrão para transfer learning.
Para mais opções de pesos pré-treinados e modelos maiores/menores, ver o github oficial do [YoloV10](https://github.com/THU-MIG/yolov10).

In [ ]:
os.chdir(os.path.join(BASE_PATH, "yolov10"))
!wget https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt

# Parâmetros básicos
Implementação de edição dos parâmetros básicos de performance do Yolo v10.<br>
Para parâmetros opcionais e outros, ver [a documentação da ultralytics](https://docs.ultralytics.com/modes/train/#train-settings) e adequar o código.<br>



In [ ]:
# Definir onde salvar os resultados de treino
#SAVE_PATH = "/content/gdrive/MyDrive/aula_yoloV10/" # salvar no drive
SAVE_PATH = os.path.join(BASE_PATH, "yolov10_run") # salvar no ambiente local

os.makedirs(SAVE_PATH, exist_ok=True)

# Definir o batch size
BATCH_SIZE = 16

# Definir o número de épocas de treino
EPOCHS = 10

# Definir o formato de imagem para rede
IM_SIZE = 640 # padrão 640, imagem menor = mais rápido, menos preciso em objetos pequenos; imagem maior = mais lento, mais preciso em objetos pequenos.

# Definir GPU a ser utilizada
GPU_USE = 0

In [ ]:
os.chdir(os.path.join(BASE_PATH, "yolov10"))
!yolo task=detect mode=train data="{BASE_PATH}/trafficsign/data.yaml" model="yolov10m.pt" epochs={EPOCHS} batch={BATCH_SIZE} imgsz={IM_SIZE} project={SAVE_PATH} device={GPU_USE}

# Gráficos de treino

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import matplotlib.ticker as ticker

metrics_img = glob(os.path.join(SAVE_PATH, 'train/*.png'))
n_img = len(metrics_img)
n_cols = 2
n_rows = int(n_img / n_cols) + (1 if n_img % n_cols != 0 else 0)
_, axs = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axs = axs.flatten()
for m, ax in zip(metrics_img, axs):
  img = plt.imread(m)
  ax.imshow(img)
  ax.xaxis.set_major_locator(ticker.NullLocator())
  ax.yaxis.set_major_locator(ticker.NullLocator())
plt.show()

# Realizar inferência de métricas no conjunto de validação (val/valid)

In [ ]:
curr_weight = os.path.join(SAVE_PATH, "train/weights/best.pt")
!yolo task=detect mode=val data="{BASE_PATH}/trafficsign/data.yaml" batch={BATCH_SIZE} device=0 model="{curr_weight}" project={SAVE_PATH} name="val"

# Visualização dos gráficos das métricas de validação

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import matplotlib.ticker as ticker

metrics_img = glob(os.path.join(SAVE_PATH, "val/*.png"))
n_img = len(metrics_img)
n_cols = 2
n_rows = int(n_img / n_cols) + (1 if n_img % n_cols != 0 else 0)
_, axs = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axs = axs.flatten()
for m, ax in zip(metrics_img, axs):
  img = plt.imread(m)
  ax.imshow(img)
  ax.xaxis.set_major_locator(ticker.NullLocator())
  ax.yaxis.set_major_locator(ticker.NullLocator())
plt.show()


# Visualizar detecções no conjunto de teste (test)

In [ ]:
curr_weight = os.path.join(SAVE_PATH, "train/weights/best.pt")
!yolo task=detect mode=predict model="{curr_weight}" source="{BASE_PATH}/trafficsign/images/test" project="{SAVE_PATH}" name="detect"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import matplotlib.ticker as ticker
import random

num_samples = 4 # numero de imagens para visualizar.
all_dets = glob(os.path.join(SAVE_PATH, "detect/*.png"))
n_cols = 2
n_rows = int(num_samples / n_cols) + (1 if num_samples % n_cols != 0 else 0)
_, axs = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axs = axs.flatten()
for m, ax in zip(random.sample(all_dets, num_samples), axs):
  img = plt.imread(m)
  ax.imshow(img)
  ax.xaxis.set_major_locator(ticker.NullLocator())
  ax.yaxis.set_major_locator(ticker.NullLocator())
plt.show()

<img src="http://lapix.ufsc.br/wp-content/uploads/2022/10/rodape-lapix.png">